
# 환경설정

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner/

/content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner


In [0]:
! pip install torch torchvision
! pip install pytorch_pretrained_bert>=0.4.0
! pip install mxnet>=1.5.0
! pip install gluonnlp #>=0.6.0
! pip install sentencepiece>=0.1.6
! pip install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
! pip install transformers
! pip install tb-nightly
! pip install future

In [11]:
! pip install konlpy

In [5]:
import torch
torch.cuda.current_device()

RuntimeError: ignored

In [0]:
torch.cuda.device(0)

In [0]:
torch.cuda.device_count()

In [0]:
torch.cuda.get_device_name(0)

In [0]:
torch.cuda.is_available()

In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import torch
from gluonnlp.data import SentencepieceTokenizer
from model.net import KobertCRF
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [0]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for i, (token_str, pred_ner_tag_str) in enumerate(zip(input_token, pred_ner_tag)):
            if i == 0 or i == len(pred_ner_tag)-1: # remove [CLS], [SEP]
                continue
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [0]:
! pip install pytorch-transformers

In [0]:
# ! python train_bert_crf.py

In [9]:
! python -m pip install tqdm

In [0]:
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')

def main(x):
    model_dir = Path('./experiments/base_model_with_crf')
    # /content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner/experiments
    model_config = Config(json_path=model_dir / 'config.json') 
    

    # load vocab & tokenizer 
    tok_path = "./ptr_lm_model/tokenizer_78b3253a26.model" 
    ptr_tokenizer = SentencepieceTokenizer(tok_path)

    with open(model_dir / "vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)
    tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=model_config.maxlen)

    # load ner_to_index.json
    with open(model_dir / "ner_to_index.json", 'rb') as f:
        ner_to_index = json.load(f)
        index_to_ner = {v: k for k, v in ner_to_index.items()}

    # model
    model = KobertCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)

    # load
    model_dict = model.state_dict()
    
    checkpoint = torch.load("/content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner/experiments/base_model_with_crf_val/best-epoch-6-step-500-acc-0.957.bin", map_location=torch.device('cpu'))
    # checkpoint = torch.load("./experiments/base_model_with_crf/best-epoch-16-step-1500-acc-0.993.bin", map_location=torch.device('cpu'))
    # checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))
    
    convert_keys = {}
    for k, v in checkpoint['model_state_dict'].items():
        new_key_name = k.replace("module.", '')
        if new_key_name not in model_dict:
            print("{} is not int model_dict".format(new_key_name))
            continue
        convert_keys[new_key_name] = v

    model.load_state_dict(convert_keys)
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)


    ner_text = []
    for input_text in tqdm(sent_tokenize(x)):
      try:
        list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([input_text])
        x_input = torch.tensor(list_of_input_ids).long()
        list_of_pred_ids = model(x_input)

        list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
        ner_text.append(decoding_ner_sentence)
      
      except:
        pass
        
    return ner_text

      # ner_texts.append(ner_text)

    # return ner_texts

### 황교안

In [0]:
import pandas as pd

In [0]:
df_hwang = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/황교안_pre.csv')
df_hwang.head(3)

In [0]:
df_hwang['kobert_ner'] = df_hwang['content'].apply(main)
df_hwang.head(3)

In [0]:
df_hwang.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/황교안_ner.csv')

### 이낙연

In [0]:
df_lee = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/이낙연_pre.csv')

In [0]:
df_lee['kobert_ner'] = df_lee['content'].apply(main)
df_lee.head()
df_lee.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/이낙연_ner.csv', index=False)

### 종로구

In [0]:
import pandas as pd

In [0]:
df_jongro = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/종로구_pre.csv')

In [0]:
df_jongro['kobert_ner'] = df_jongro['content'].apply(main)
df_jongro.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/종로구_ner.csv', index=False)